# --------------------------------------------------------
# Load the Prepped Data into
- lead_time_data.csv
- sales_forecast_data.csv

both files need to go through some final processing before they are ready to train models

[Data Prep File](../../src/DataPrep/data_prep.py)

2 functions to read, both are noterized
-   create_lead_time_data
-   create_sales_forecast_data

### ------------------------------------------------------------------------------------------------

Lead_time_data.csv:
-   Column Name     | Description
-   PODate          | Date the item was ordered (purchases_df) (yyyy-mm-dd)
-   ReceivingDate   | Date the item was received (purchases_df) (yyyy-mm-dd)
-   LeadTimeDays    | Target = ReceivingDate - PODate (in days)
-   ItemID          | Item identifier (purchases_df)
-   Description     | Item description (inventory_df)
-   StoreID         | Store identifier (purchases_df)
-   Location        | Store location (stores_df)
-   Quantity        | Quantity ordered (purchases_df)
-   Week            | Week number of the PODate     (optional feature)
-   Month           | Month of PODate               (optional feature)
-   Day             | Day of week PODate was placed (optional feature)

### ------------------------------------------------------------------------------------------------

Sales_forecast.csv:
-   Column Name     | Description
-   SalesDate       | Date of sale (daily or weekly aggregated)
-   ItemID          | Item identifier
-   StoreID         | Store identifier
-   SalesQuantity   | Target: quantity sold
-   Lag_1           | Sales one day/week before 
-   Lag_7           | Sales one week before 
-   RollingAvg_7    | 7-day rolling average 
-   Month           | Month of sale
-   DayOfWeek       | Day of week of sale
-   StockOnHand     | (optional) On-hand inventory if available
-   ReceivedQty     | Quantity received in last x days (optional)

Lag features (1 day/week), temporal / short term trends.
-   Lag features are used to capture the temporal dependencies in the data, they are the values of the target variable 
-   (SalesQuantity) from previous time steps.

Rolling average feature (7-day rolling average)
-   Smooths out short-term fluctuations and highlights longer-term trends or cycles.
-   Gives the model trend signal and helps to reduce noise in the data.

In [2]:
import sys
import os

# Add the root directory to the system path
# This is necessary to import modules from the root directory
sys.path.append("../..")
from src.DataPrep.data_prep import create_lead_time_data
from src.DataPrep.data_prep import create_sales_forecast_data

# run the 2 functions to create the data
lead_time_df = create_lead_time_data()
sales_forecast_df = create_sales_forecast_data()

# print the dataframes to check if they are created correctly
print("Lead Time DataFrame:")
display(lead_time_df.head())
print("Sales Forecast DataFrame:")
display(sales_forecast_df.head())


Lead Time DataFrame:


,PODate,ReceivingDate,LeadTimeDays,ItemID,Description,StoreID,Location,Quantity,Week,Month,Day
0,2015-12-22,2016-01-01,10,5255,TGI Fridays Ultimte Mudslide,1,HARDERSFIELD,6,52,12,1
1,2015-12-20,2016-01-01,12,8358,Bacardi 151 Proof,1,HARDERSFIELD,12,51,12,6
2,2015-12-20,2016-01-01,12,4233,Castillo Silver Label Rum,1,HARDERSFIELD,23,51,12,6
3,2015-12-20,2016-01-01,12,3830,Grey Goose L'Orange Vodka,1,HARDERSFIELD,6,51,12,6
4,2015-12-20,2016-01-01,12,4670,Bacardi Dragon Berry Rum,1,HARDERSFIELD,11,51,12,6


Sales Forecast DataFrame:


,SalesDate,ItemID,StoreID,SalesQuantity,Lag_1,Lag_7,RollingAvg_7,Month,DayOfWeek
0,2016-01-01,1004,1,1,0.0,0.0,0.0,1,4
1,2016-01-02,1004,1,2,1.0,0.0,0.0,1,5
2,2016-01-03,1004,1,1,2.0,0.0,0.0,1,6
3,2016-01-08,1004,1,1,1.0,0.0,0.0,1,4
4,2016-01-09,1005,1,2,0.0,0.0,0.0,1,5
